In [ ]:
#Spark session

In [4]:
from pyspark.sql import SparkSession


In [5]:
spark = SparkSession.builder\
                    .appName("Analyzing London crime data")\
                    .getOrCreate()

In [ ]:
# read

In [6]:
data =spark.read\
            .format("csv")\
            .option("header", "true")\
            .load("/data/pluralsight_spark2/02/demos/datasets/london_crime_by_lsoa.csv")

In [8]:
data.printSchema()

root
 |-- lsoa_code: string (nullable = true)
 |-- borough: string (nullable = true)
 |-- major_category: string (nullable = true)
 |-- minor_category: string (nullable = true)
 |-- value: string (nullable = true)
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)



In [9]:
data.count()

13490604

In [10]:
data.limit(5).show()

+---------+----------+--------------------+--------------------+-----+----+-----+
|lsoa_code|   borough|      major_category|      minor_category|value|year|month|
+---------+----------+--------------------+--------------------+-----+----+-----+
|E01001116|   Croydon|            Burglary|Burglary in Other...|    0|2016|   11|
|E01001646| Greenwich|Violence Against ...|      Other violence|    0|2016|   11|
|E01000677|   Bromley|Violence Against ...|      Other violence|    0|2015|    5|
|E01003774| Redbridge|            Burglary|Burglary in Other...|    0|2016|    3|
|E01004563|Wandsworth|             Robbery|   Personal Property|    0|2008|    6|
+---------+----------+--------------------+--------------------+-----+----+-----+



In [11]:
data.dropna()

DataFrame[lsoa_code: string, borough: string, major_category: string, minor_category: string, value: string, year: string, month: string]

In [13]:
data = data.drop("lsoa_code")
data.show()

+--------------------+--------------------+--------------------+-----+----+-----+
|             borough|      major_category|      minor_category|value|year|month|
+--------------------+--------------------+--------------------+-----+----+-----+
|             Croydon|            Burglary|Burglary in Other...|    0|2016|   11|
|           Greenwich|Violence Against ...|      Other violence|    0|2016|   11|
|             Bromley|Violence Against ...|      Other violence|    0|2015|    5|
|           Redbridge|            Burglary|Burglary in Other...|    0|2016|    3|
|          Wandsworth|             Robbery|   Personal Property|    0|2008|    6|
|              Ealing|  Theft and Handling|         Other Theft|    0|2012|    5|
|              Ealing|Violence Against ...|    Offensive Weapon|    0|2010|    7|
|            Hounslow|             Robbery|   Personal Property|    0|2013|    4|
|              Newham|     Criminal Damage|Criminal Damage T...|    0|2013|    9|
|              S

In [14]:
total_boroughs = data.select('borough')\
                        .distinct()
total_boroughs.show()

+--------------------+
|             borough|
+--------------------+
|             Croydon|
|          Wandsworth|
|              Bexley|
|             Lambeth|
|Barking and Dagenham|
|              Camden|
|           Greenwich|
|              Newham|
|       Tower Hamlets|
|            Hounslow|
|              Barnet|
|              Harrow|
|Kensington and Ch...|
|           Islington|
|               Brent|
|            Haringey|
|             Bromley|
|              Merton|
|         Westminster|
|             Hackney|
+--------------------+
only showing top 20 rows



In [15]:
total_boroughs.count()

33

In [ ]:
#Filters

In [17]:
hackney_data = data.filter(data['borough'] == "Hackney")
hackney_data.show()

+-------+--------------------+--------------------+-----+----+-----+
|borough|      major_category|      minor_category|value|year|month|
+-------+--------------------+--------------------+-----+----+-----+
|Hackney|     Criminal Damage|Criminal Damage T...|    0|2011|    6|
|Hackney|Violence Against ...|          Harassment|    1|2013|    2|
|Hackney|     Criminal Damage|Other Criminal Da...|    0|2011|    7|
|Hackney|Violence Against ...|        Wounding/GBH|    0|2013|   12|
|Hackney|  Theft and Handling|  Other Theft Person|    0|2016|    8|
|Hackney|            Burglary|Burglary in a Dwe...|    2|2008|    5|
|Hackney|             Robbery|   Business Property|    0|2016|    7|
|Hackney|  Theft and Handling|Theft/Taking of P...|    0|2009|   12|
|Hackney|               Drugs|    Drug Trafficking|    0|2014|    4|
|Hackney|  Theft and Handling|Handling Stolen G...|    0|2014|    6|
|Hackney|            Burglary|Burglary in Other...|    0|2008|   12|
|Hackney|Violence Against ...| Ass

# isin filter

In [18]:
data_2015_2016 = data.filter(data['year'].isin(["2015", "2016"]))
data_2015_2016.sample(fraction=0.1).show()

+--------------------+--------------------+--------------------+-----+----+-----+
|             borough|      major_category|      minor_category|value|year|month|
+--------------------+--------------------+--------------------+-----+----+-----+
|           Redbridge|            Burglary|Burglary in Other...|    0|2016|    3|
|            Hounslow|     Criminal Damage|Criminal Damage T...|    0|2015|    2|
|              Newham|     Criminal Damage|Criminal Damage T...|    0|2015|    1|
|             Croydon|             Robbery|   Personal Property|    0|2016|    1|
|             Lambeth|               Drugs|    Drug Trafficking|    0|2015|    9|
|           Redbridge|               Drugs| Possession Of Drugs|    2|2016|   11|
|         Westminster|  Theft and Handling|Theft From Motor ...|    6|2016|    8|
|              Harrow|               Drugs| Possession Of Drugs|    0|2016|   11|
|           Redbridge|            Burglary|Burglary in Other...|    0|2015|    2|
|Hammersmith and

In [21]:
data_2014_onwards = data.filter(data['year'] >= 2014)
data_2014_onwards.sample(fraction=0.1).show()

+--------------------+--------------------+--------------------+-----+----+-----+
|             borough|      major_category|      minor_category|value|year|month|
+--------------------+--------------------+--------------------+-----+----+-----+
|              Newham|  Theft and Handling|Theft/Taking of P...|    0|2016|    3|
|              Newham|Violence Against ...|      Other violence|    0|2015|    3|
|            Hounslow|Violence Against ...|    Offensive Weapon|    1|2015|   10|
|              Ealing|             Robbery|   Personal Property|    0|2014|    9|
|Kingston upon Thames|Other Notifiable ...|      Going Equipped|    0|2014|    5|
|              Newham|Violence Against ...|          Harassment|    0|2014|    1|
|              Ealing|Violence Against ...|      Common Assault|    1|2015|   12|
|       Tower Hamlets|  Theft and Handling|Handling Stolen G...|    0|2015|   10|
|Richmond upon Thames|  Theft and Handling|Theft/Taking of P...|    0|2016|   10|
|             Ha